In [ ]:
import pandas as pd

from Base import *

In [ ]:
session = ff1.get_event(2022, 'Monza').get_race()
session.load()
alllaps = session.laps
alllaps.columns

In [ ]:
groupedLaps = alllaps.groupby('DriverNumber')

lap_list = []
number = 1

with pd.ExcelWriter('2022 Italian Grand Prix Lap Deltas.xlsx') as writer:
    for group_name, group_df in groupedLaps:
        avg_lap = group_df.loc[group_df['IsAccurate'], 'LapTime'].mean()
        for lap in group_df.itertuples(index=False):
            if not lap.LapTime or lap.IsAccurate != True:
                lap_list.append('NaN')
            else:
                laptime = lap.LapTime - avg_lap
                lap_list.append(laptime)
        delta_col = pd.Series(lap_list, name = 'DeltaTime')
        group_df = pd.concat([group_df, delta_col], axis = 1)

        ax = plt.subplot(20, 1, number)
        ax = plt.gca()
        ax.plot(group_df['LapNumber'], group_df['DeltaTime'], linewidth = 10)
        ax.set(xlabel='Lap Number', ylabel='Delta Time', title=f"{group_name} Laps")
        #group_df.to_excel(writer, sheet_name = group_name, index = False)
        number += 1
    plt.savefig('test.png')

In [ ]:
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (120,600)
plt.rcParams['text.color'] = 'b'
plt.rcParams['legend.labelcolor'] = 'w'
plt.rcParams['font.size'] = 40
ax = plt.subplot()



In [ ]:
with pd.ExcelWriter('2022 Italian Grand Prix Lap Deltas.xlsx') as writer:
    for group_name, group_df in groupedLaps:
        for row in group_df.itertuples(index = True):
            index = row.Index

            laptimevalue = row.LapTime
            newvalue = laptimevalue + pd.Timedelta(seconds = (0.052 * index))
            group_df.at[index, 'LapTime'] = newvalue

            group_df.to_excel(writer, sheet_name=group_name)

In [ ]:
FUELDELTA = 0.052

alllaps = alllaps.pick_accurate().pick_track_status('1')

for row in alllaps.itertuples(index = True):
    index = row.Index

    laptimevalue = row.LapTime
    newvalue = laptimevalue + pd.Timedelta(seconds = (FUELDELTA * (row.LapNumber - 1)))
    alllaps.at[index, 'FuelAdjustedLapTime'] = newvalue

In [ ]:
from scipy.stats import linregress

groupedLaps = alllaps.groupby('Driver')
regression_df = pd.DataFrame(columns=['Compound', 'Slope', 'Intercept', 'R Value', 'P Value', 'Standard Error', 'No of Laps', 'Laps Till End', 'Team'])

for driver_name, driver_df in groupedLaps:

    groupedStints = driver_df.groupby('Stint')

    for stint_name, stint_df in groupedStints:
        stint_df['NumericalTime'] = stint_df['FuelAdjustedLapTime'].dt.total_seconds()

        no_laps = len(stint_df)
        laps_till_end = alllaps['LapNumber'].max() - stint_df['LapNumber'].min()
        team = stint_df['Team'].iloc[0]
        compound = stint_df['Compound'].iloc[0]

        slope, intercept, r_value, p_value, std_err = linregress(stint_df['LapNumber'], stint_df['NumericalTime'])

        regression_df.loc[f'{driver_name} -  Stint {stint_name}'] = [compound, slope, intercept, r_value, p_value, std_err, no_laps, laps_till_end, team]

In [ ]:
regression_df

In [ ]:
alllaps.to_excel('test.xlsx')